# Hyperasの練習

*20170715 Mitsuhiro Ito*


* (https://github.com/maxpumperla/hyperas/tree/master/examples)


---

In [1]:
from __future__ import print_function
import sys
from utils import save_model_viz, save_weights, save_hist, plot_hist
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from keras import optimizers

Using TensorFlow backend.


In [2]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import uniform, choice, conditional
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [3]:
def data():
    train_gan = ImageDataGenerator(rescale=1.0 / 255,
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    valid_gan = ImageDataGenerator(rescale=1.0 / 255,
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    train_generator = train_gan.flow_from_directory('/home/dl/mitsu-work/data/train/',
        target_size=(150, 150),
        batch_size=16,
        class_mode='binary')

    valid_generator = valid_gan.flow_from_directory('/home/dl/mitsu-work/data/valid/',
        target_size=(150, 150),
        batch_size=16,
        class_mode='binary')
    
    
    
    return train_generator, valid_generator

In [4]:
def model(train_generator, valid_generator):

    model = Sequential()
    model.add(Conv2D(64, (3, 3), input_shape=(150, 150, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    adam = optimizers.Adam(lr={{choice([0.001, 0.0005, 0.0001])}}, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
 
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    batch_size = 16
    
              

 
    model.fit_generator(train_generator, 
                    steps_per_epoch = 2000 // batch_size, 
                    epochs=20, 
                    verbose=1, 
                    validation_data = valid_generator,
                    validation_steps= 800 // batch_size)
    
    
    score, acc = model.evaluate_generator(generator=valid_generator, 
                                      steps= 800 // batch_size)
    
    print(score, acc)
    
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [5]:
train_generator, valid_generator = data()

best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=10,
                                      trials=Trials(),
                                      notebook_name='20170714_hyperas6' )
print("Evalutation of best performing model:")
print(best_model.evaluate_generator(valid_generator, steps=64))
print(best_run)



Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


SyntaxError: invalid syntax (<unknown>, line 108)